In [1]:
import numpy as np
import pandas as pd
import warnings
import time
import os
import sys
%matplotlib inline
from scipy import signal
from scipy.signal import butter, lfilter
import matplotlib.pyplot as plt
import math
import pickle
import statistics
import random
from sklearn.metrics.pairwise import cosine_similarity
from math import log
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import sktime as sktime
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sktime.datasets import load_from_tsfile_to_dataframe
from sktime.alignment.dtw_python import AlignerDTWfromDist
from sktime.dists_kernels.scipy_dist import ScipyDist
from sktime.dists_kernels.compose_from_align import DistFromAligner
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import RidgeClassifierCV
from sktime.transformations.panel.rocket import Rocket
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.classification.hybrid import HIVECOTEV2
from sktime.classification.interval_based import CanonicalIntervalForest
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Dictionary saved to hb_sepsis_dtwknn.pickle
Dictionary saved to healthy_sepsis_dtwknn.pickle


### Load normal and abnormal datasets

In [2]:
final_normal_df = pd.read_pickle('normal_public_sepsis.pkl')
final_abnormal_df = pd.read_pickle('abnormal_public_sepsis.pkl')

In [3]:
abnormal_indices = (final_abnormal_df['Subject']).unique()
abnormal_no = (final_abnormal_df['Subject']).nunique()

'Number of abnormal subjects:'

175

In [4]:
# Specify the file path of the saved pkl file
file_path = 'final_bins.pkl'

# Load the dictionary from the pkl file
with open(file_path, 'rb') as file:
    final_numbers_dictionary = pickle.load(file)

'Dictionary loaded from pickle file:'

### Function to find the length of the time series

In [5]:
def time_series_length_finding(df):
    train_1_abnormal = df.copy()
    train_1_ab_sub = (train_1_abnormal['Subject'].unique()).tolist()
    row_count = []
    for i in train_1_ab_sub:
        train_1_abnormal_new = train_1_abnormal[(train_1_abnormal['Subject']== i)]
        no_rows = train_1_abnormal_new.shape[0]
        row_count.append(no_rows)
    display("Avg length:",statistics.mean(row_count))
    avg_len_ts = int(statistics.mean(row_count))
    return(avg_len_ts)

In [6]:
# Define a function to z-normalize a column
def z_normalize(column):
    mean = column.mean()
    std = column.std()
    z_normalized = (column - mean) / std
    return z_normalized

In [7]:
def preprocessing_normal_df(df,desired_length):
    train_2_normal = df.copy()
    df_train_3 = pd.DataFrame(columns = train_2_normal.columns)
    num_dim = df_train_3.shape[1]
    display(num_dim)
    df_train_3.columns = range(len(df_train_3.columns))
    train_2_sub = (train_2_normal['Subject'].unique()).tolist()
    m = 0
    for i in train_2_sub:
        train_2_normal_new = train_2_normal[(train_2_normal['Subject']== i)]
        df_row_count = train_2_normal_new.shape[0]
        if df_row_count < desired_length:
            rows_to_add = desired_length - df_row_count
            train_2_normal_new.tail()
            last_row = train_2_normal_new.ffill().iloc[[-1]]  # Extract the last row
            new_rows_df = pd.concat([last_row] * rows_to_add, ignore_index=True)
            train_2_normal_initial = pd.concat([train_2_normal_new, new_rows_df], ignore_index=True)
            train_2_normal_initial_no_sub = train_2_normal_initial.drop(columns='Subject')
            train_2_normal_final = z_normalize(train_2_normal_initial_no_sub)
            train_2_normal_final.insert(0, 'Subject', i)
        else:
            train_2_normal_initial = train_2_normal_new.iloc[:desired_length,:]
            train_2_normal_initial_no_sub = train_2_normal_initial.drop(columns='Subject')
            train_2_normal_final = z_normalize(train_2_normal_initial_no_sub)
            train_2_normal_final.insert(0, 'Subject', i)
            
        train_2_normal_final.columns = range(len(train_2_normal_final.columns))
        
        df_train_3.at[m,0] = i
        j = 0
        while j < num_dim:
            df_train_3.at[m,j] = train_2_normal_final[j]
            j += 1
        m += 1    
    df_train_3['class'] = 'normal'
    df_normal_baseline = df_train_3.iloc[:,1:]
    return(df_normal_baseline)

In [8]:
def preprocessing_dataframe(df,desired_length):
    train_1_abnormal = df.copy()
    df_train_2 = pd.DataFrame(columns = train_1_abnormal.columns)
    num_dim_ab = df_train_2.shape[1]
    df_train_2.columns = range(len(df_train_2.columns))
    train_1_ab_sub = (train_1_abnormal['Subject'].unique()).tolist()
    m = 0
    for i in train_1_ab_sub:
        train_1_abnormal_new = train_1_abnormal[(train_1_abnormal['Subject']== i)]
        df_row_count = train_1_abnormal_new.shape[0]
        if df_row_count < desired_length:
            rows_to_add = desired_length - df_row_count
            last_row = train_1_abnormal_new.ffill().iloc[[-1]]  # Extract the last row
            new_rows_df = pd.concat([last_row] * rows_to_add, ignore_index=True)
            train_1_abnormal_initial = pd.concat([train_1_abnormal_new, new_rows_df], ignore_index=True)
            train_1_abnormal_initial_no_sub = train_1_abnormal_initial.drop(columns='Subject')
            train_1_abnormal_final = z_normalize(train_1_abnormal_initial_no_sub)
            train_1_abnormal_final.insert(0, 'Subject', i)
        else:
            train_1_abnormal_initial = train_1_abnormal_new.iloc[:desired_length,:]
            train_1_abnormal_initial_no_sub = train_1_abnormal_initial.drop(columns='Subject')
            train_1_abnormal_final = z_normalize(train_1_abnormal_initial_no_sub)
            train_1_abnormal_final.insert(0, 'Subject', i)
        train_1_abnormal_final.columns = range(len(train_1_abnormal_final.columns))
        
        df_train_2.at[m,0] = i
        j = 0
        while j < num_dim_ab:
            df_train_2.at[m,j] = train_1_abnormal_final[j]
            j += 1
        m += 1    
    # display(df_train_2)
    df_train_2['class'] = 'abnormal'
    df_final = df_train_2.iloc[:,1:]
    return(df_final)

In [9]:
def ts_classifier(df_class_1):
    accuacy_list_1 = []
    precision_list_1 = []
    recall_list_1 = []
    f1_list_1 = []

    X_1 = df_class_1.iloc[:,:-1]
    y_1 = df_class_1.iloc[:,-1]
    
    display("X_1:",X_1)

    for i in range(1,31):
        X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=0.2, random_state=i,stratify = y_1) 
        dtw_knn = KNeighborsTimeSeriesClassifier(distance="dtw")
        dtw_knn.fit(X_train_1, y_train_1)
        y_pred_1 = dtw_knn.predict(X_test_1)

        pos_label = 'abnormal'

        acc_1 = accuracy_score(y_test_1, y_pred_1)
        accuacy_list_1.append(acc_1)

        prec_1 = precision_score(y_test_1, y_pred_1, pos_label=pos_label)
        precision_list_1.append(prec_1)

        recall_1 = recall_score(y_test_1, y_pred_1, pos_label=pos_label)
        recall_list_1.append(recall_1)

        f1_1 = f1_score(y_test_1, y_pred_1, pos_label=pos_label)
        f1_list_1.append(f1_1)
        display("Completed",i)

    acc_mean_1 = np.mean(accuacy_list_1)
    acc_std_dev_1 = np.std(accuacy_list_1)

    prec_mean_1 = np.mean(precision_list_1)
    prec_std_dev_1 = np.std(precision_list_1)

    recall_mean_1 = np.mean(recall_list_1)
    recall_std_dev_1 = np.std(recall_list_1)

    f1_mean_1 = np.mean(f1_list_1)
    f1_std_dev_1 = np.std(f1_list_1)
    
    return(acc_mean_1,acc_std_dev_1,prec_mean_1,prec_std_dev_1,recall_mean_1,recall_std_dev_1,f1_mean_1,f1_std_dev_1)

In [10]:
# Keys to combine
key_to_combine1 = 'sample_numbers_list_0.0_to_0.1'
key_to_combine2 = 'sample_numbers_list_0.1_to_0.2'
# Create a new dictionary with combined values
hb_dict = {
    'hb_key': final_numbers_dictionary[key_to_combine1] + final_numbers_dictionary[key_to_combine2]
}

In [11]:
classifier_dict_hb = {}
key = 'hb_key'
normal_sample_numbers = [] 
normal_sample_numbers.extend(hb_dict[key])
normal_sample_length = len(normal_sample_numbers)
display(normal_sample_length)

if len(normal_sample_numbers) <= abnormal_no:
    df_normal = final_normal_df[final_normal_df['Subject'].isin(normal_sample_numbers)]

    random.seed(42)
    abnormal_sample_numbers = random.sample(abnormal_indices.tolist(), normal_sample_length)
    df_abnormal = final_abnormal_df[final_abnormal_df['Subject'].isin(abnormal_sample_numbers)]

    desired_length_for_ts = time_series_length_finding(df_abnormal)

    df_normal_preprocessed = preprocessing_normal_df(df_normal,desired_length_for_ts)
    df_abnormal_preprocessed = preprocessing_dataframe(df_abnormal,desired_length_for_ts)

    df_classifier = (pd.concat([df_normal_preprocessed,df_abnormal_preprocessed]).reset_index()).drop(columns='index')
    display("df_classifier:",df_classifier.head())
    acc_mean,acc_std_dev,prec_mean,prec_std_dev,recall_mean,recall_std_dev,f1_mean,f1_std_dev = ts_classifier(df_classifier)

    classifier_dict_hb[key] = {'acc_mean': acc_mean,'acc_std_dev': acc_std_dev,'prec_mean': prec_mean,'prec_std_dev': prec_std_dev,'recall_mean': recall_mean,'recall_std_dev': recall_std_dev,'f1_mean': f1_mean,'f1_std_dev': f1_std_dev}

else:
    df_abnormal = final_abnormal_df

    random.seed(42)
    random_sample_numbers = random.sample(normal_sample_numbers, abnormal_no)
    df_normal = final_normal_df[final_normal_df['Subject'].isin(random_sample_numbers)]

    desired_length_for_ts = time_series_length_finding(df_abnormal)

    df_normal_preprocessed = preprocessing_normal_df(df_normal,desired_length_for_ts)
    df_abnormal_preprocessed = preprocessing_dataframe(df_abnormal,desired_length_for_ts)
    
    df_classifier = (pd.concat([df_normal_preprocessed,df_abnormal_preprocessed]).reset_index()).drop(columns='index')
    acc_mean,acc_std_dev,prec_mean,prec_std_dev,recall_mean,recall_std_dev,f1_mean,f1_std_dev = ts_classifier(df_classifier)

    classifier_dict_hb[key] = {'acc_mean': acc_mean,'acc_std_dev': acc_std_dev,'prec_mean': prec_mean,'prec_std_dev': prec_std_dev,'recall_mean': recall_mean,'recall_std_dev': recall_std_dev,'f1_mean': f1_mean,'f1_std_dev': f1_std_dev}

72

'Avg length:'

106.48611111111111

6

'df_classifier:'

...

28

'Completed'

29

'Completed'

30

In [12]:
classifier_dict_hb

{'hb_key': {'acc_mean': 0.7735632183908049,
  'acc_std_dev': 0.07270512460169411,
  'prec_mean': 0.7907793831555441,
  'prec_std_dev': 0.09751406083999678,
  'recall_mean': 0.7561904761904763,
  'recall_std_dev': 0.11175369742826806,
  'f1_mean': 0.7660405865027307,
  'f1_std_dev': 0.07710040220398275}}

In [13]:
import pickle

# Specify the file path where you want to save the pickle file
pickle_file_path = 'hb_sepsis_dtwknn.pickle'

# Save the dictionary to a pickle file
with open(pickle_file_path, 'wb') as file:
    pickle.dump(classifier_dict_hb, file)

print(f"Dictionary saved to {pickle_file_path}")

In [14]:
all_values = [value for sublist in final_numbers_dictionary.values() for value in sublist]
healthy_dict = {'healthy_key': all_values}

{'healthy_key': [112,
  360,
  143,
  415,
  413,
  23,
  374,
  32,
  34,
  42,
  68,
  90,
  119,
  121,
  137,
  178,
  180,
  196,
  214,
  241,
  242,
  249,
  257,
  265,
  277,
  279,
  315,
  325,
  340,
  357,
  365,
  369,
  406,
  407,
  426,
  429,
  441,
  455,
  458,
  470,
  493,
  14,
  288,
  20,
  33,
  35,
  51,
  52,
  60,
  63,
  67,
  73,
  144,
  167,
  176,
  205,
  228,
  235,
  250,
  296,
  313,
  323,
  337,
  362,
  396,
  409,
  419,
  440,
  464,
  467,
  486,
  489,
  378,
  280,
  50,
  3,
  5,
  41,
  43,
  70,
  117,
  123,
  139,
  164,
  220,
  225,
  244,
  247,
  262,
  289,
  320,
  334,
  344,
  350,
  363,
  381,
  393,
  394,
  417,
  444,
  474,
  480,
  487,
  496,
  72,
  457,
  7,
  27,
  30,
  69,
  79,
  100,
  138,
  150,
  160,
  163,
  204,
  232,
  245,
  300,
  331,
  336,
  341,
  352,
  377,
  402,
  408,
  435,
  481,
  500,
  380,
  473,
  194,
  6,
  21,
  36,
  37,
  77,
  103,
  109,
  125,
  128,
  132,
  135,
  149,
  187,


In [15]:
classifier_dict_healthy = {}
key = 'healthy_key'
normal_sample_numbers = [] 
normal_sample_numbers.extend(healthy_dict[key])
normal_sample_length = len(normal_sample_numbers)
display(normal_sample_length)

if len(normal_sample_numbers) <= abnormal_no:
    df_normal = final_normal_df[final_normal_df['Subject'].isin(normal_sample_numbers)]
    display(df_normal.head())

    random.seed(42)
    abnormal_sample_numbers = random.sample(abnormal_indices.tolist(), normal_sample_length)
    df_abnormal = final_abnormal_df[final_abnormal_df['Subject'].isin(abnormal_sample_numbers)]
    display(df_abnormal.head())

    desired_length_for_ts = time_series_length_finding(df_abnormal)

    df_normal_preprocessed = preprocessing_normal_df(df_normal,desired_length_for_ts)
    df_abnormal_preprocessed = preprocessing_dataframe(df_abnormal,desired_length_for_ts)

    df_classifier = (pd.concat([df_normal_preprocessed,df_abnormal_preprocessed]).reset_index()).drop(columns='index')
    acc_mean,acc_std_dev,prec_mean,prec_std_dev,recall_mean,recall_std_dev,f1_mean,f1_std_dev = ts_classifier(df_classifier)

    classifier_dict_healthy[key] = {'acc_mean': acc_mean,'acc_std_dev': acc_std_dev,'prec_mean': prec_mean,'prec_std_dev': prec_std_dev,'recall_mean': recall_mean,'recall_std_dev': recall_std_dev,'f1_mean': f1_mean,'f1_std_dev': f1_std_dev}

else:
    df_abnormal = final_abnormal_df

    random.seed(42)
    random_sample_numbers = random.sample(normal_sample_numbers, abnormal_no)
    df_normal = final_normal_df[final_normal_df['Subject'].isin(random_sample_numbers)]
    display(df_normal)

    desired_length_for_ts = time_series_length_finding(df_abnormal)

    df_normal_preprocessed = preprocessing_normal_df(df_normal,desired_length_for_ts)
    df_abnormal_preprocessed = preprocessing_dataframe(df_abnormal,desired_length_for_ts)

    df_classifier = (pd.concat([df_normal_preprocessed,df_abnormal_preprocessed]).reset_index()).drop(columns='index')
    acc_mean,acc_std_dev,prec_mean,prec_std_dev,recall_mean,recall_std_dev,f1_mean,f1_std_dev = ts_classifier(df_classifier)

    classifier_dict_healthy[key] = {'acc_mean': acc_mean,'acc_std_dev': acc_std_dev,'prec_mean': prec_mean,'prec_std_dev': prec_std_dev,'recall_mean': recall_mean,'recall_std_dev': recall_std_dev,'f1_mean': f1_mean,'f1_std_dev': f1_std_dev}

502

,Subject,HR,MAP,DBP,SBP,Resp
0,11,90.0,70.0,53.0,111.0,24.0
1,11,90.0,70.0,53.0,111.0,24.0
2,11,90.0,80.0,60.0,121.0,24.0
3,11,81.0,67.0,52.0,99.0,12.0
4,11,83.0,66.0,51.0,99.0,14.0
...,...,...,...,...,...,...
52,506,104.0,89.0,69.0,136.0,28.0
53,506,100.0,83.0,65.0,121.0,19.0
54,506,91.0,74.0,59.0,102.0,14.0
55,506,90.0,66.0,61.0,101.0,16.0


'Avg length:'

108.59428571428572

6

...

28

'Completed'

29

'Completed'

30

In [16]:
classifier_dict_healthy

{'healthy_key': {'acc_mean': 0.6852380952380952,
  'acc_std_dev': 0.040319022591903675,
  'prec_mean': 0.708280410918097,
  'prec_std_dev': 0.04445692992991904,
  'recall_mean': 0.6352380952380954,
  'recall_std_dev': 0.08465482803300149,
  'f1_mean': 0.6662852241287144,
  'f1_std_dev': 0.053101037342524714}}

In [17]:
import pickle

# Specify the file path where you want to save the pickle file
pickle_file_path = 'healthy_sepsis_dtwknn.pickle'

# Save the dictionary to a pickle file
with open(pickle_file_path, 'wb') as file:
    pickle.dump(classifier_dict_healthy, file)

print(f"Dictionary saved to {pickle_file_path}")